# Object Oriented Languages and Environments – Final Project - Study Hours Tracker
Rudainah Khaled – B2180.060004 – Software Development – IAU – 2024

In [1]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import matplotlib.pyplot as plt

In [2]:
class StudyHoursTracker:
    def __init__(self, root):
        self.root = root
        self.root.title("Study Hours Tracker - Rudainah's App")
        
        self.setup_database()

        self.login_frame = tk.Frame(self.root)
        self.main_frame = tk.Frame(self.root)
        self.subjects_frame = tk.Frame(self.root)
        
        self.user_id = None

        self.create_login_widgets()
        self.create_main_widgets()
        self.create_subject_widgets()
        
        self.show_welcome_message()
        self.show_login()

    def setup_database(self):
        conn = sqlite3.connect('study_hours.db')
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS Users
                     (user_id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT, password TEXT)''')
        c.execute('''CREATE TABLE IF NOT EXISTS StudyRecords
                     (record_id INTEGER PRIMARY KEY AUTOINCREMENT, user_id INTEGER, grade INTEGER, subject TEXT, user_hours INTEGER, system_hours INTEGER,
                      user_avg REAL, system_avg REAL, avg_result TEXT, summary_result TEXT)''')
        conn.commit()
        conn.close()

    def create_login_widgets(self):
        tk.Label(self.login_frame, text="Username:").pack()
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.pack()
        tk.Label(self.login_frame, text="Password:").pack()
        self.password_entry = tk.Entry(self.login_frame, show='*')
        self.password_entry.pack()
        tk.Button(self.login_frame, text="Login", command=self.login).pack()
        tk.Button(self.login_frame, text="Register", command=self.register).pack()

    def create_main_widgets(self):
        self.grade_var = tk.StringVar()

        tk.Label(self.main_frame, text="Choose Your Grade (1, 2, 3, 4):").pack()
        grade_entry = tk.Entry(self.main_frame, textvariable=self.grade_var)
        grade_entry.pack()
        tk.Button(self.main_frame, text="Show Subjects", command=self.show_subjects).pack()
        tk.Button(self.main_frame, text="Show Summary", command=self.show_summary).pack()
        tk.Button(self.main_frame, text="Logout", command=self.logout).pack()

    def create_subject_widgets(self):
        self.subject_dict = [
            {'INTRODUCTION TO SOFTWARE DEVELOPMENT I': 28, 'MATHEMATICAL FOUNDATIONS OF COMPUTING': 21, 'SOURCE CODE EDITORS AND COMPILERS': 14, 'PROGRAM DESIGN I': 21, 'COMPUTER DESIGN': 12},
            {'WEB PROGRAMMING AND CONTENT MANAGEMENT SYSTEMS': 28, 'OPERATING SYSTEMS DESIGN AND IMPLEMENTATION': 21, 'INTRODUCTION TO SOFTWARE DEVELOPMENT II': 14, 'ENGLISH FOR SOFTWARE DEVELOPERS II': 7},
            {'STATISTICS': 28, 'GAME DEVELOP': 21, 'E-COMMERCE': 14, 'STRUCTURES': 7, 'RESPONSIBILITY': 21},
            {'SYSTEMS': 28, 'GAME DEVELOP': 21, 'OOP': 14, 'COMPUTING': 7, 'DATABASE': 21}
        ]
        self.user_hour_entries = []

    def show_login(self):
        self.login_frame.pack()
        self.main_frame.pack_forget()
        self.subjects_frame.pack_forget()

    def show_main(self):
        self.login_frame.pack_forget()
        self.main_frame.pack()
        self.subjects_frame.pack()

    def show_subjects(self):
        for widget in self.subjects_frame.winfo_children():
            widget.destroy()

        self.user_hour_entries = []

        grade_input = int(self.grade_var.get())
        for key in self.subject_dict[grade_input-1]:
            tk.Label(self.subjects_frame, text=f"{key}:").pack()
            entry = tk.Entry(self.subjects_frame)
            entry.pack()
            self.user_hour_entries.append(entry)

    def avg(self, hours):
        total = sum(hours)
        return total / len(hours)

    def avg_report(self, user_avg, system_avg):
        if user_avg > system_avg:
            return "You are above the average"
        elif user_avg == system_avg:
            return "Your average is perfectly aligned"
        else:
            return "You are below the average"

    def summary_report(self, user_hour, system_hour):
        if system_hour > user_hour:
            if system_hour - user_hour > 5:
                return "You should put more effort on this subject because you are more than 5 hours behind this week."
            else:
                return "You did good on this subject but you still need more than 5 hours ahead, try harder."
        else:
            return "Well done!! You did great this week on this subject, Keep up the good work."

    def show_summary(self):
        grade_input = int(self.grade_var.get())
        user_hours_list = [int(entry.get()) for entry in self.user_hour_entries]
        system_hours_list = [self.subject_dict[grade_input-1][key] for key in self.subject_dict[grade_input-1]]

        user_avg = self.avg(user_hours_list)
        system_avg = self.avg(system_hours_list)

        avg_result = self.avg_report(user_avg, system_avg)
        summary_results = [self.summary_report(user_hours_list[i], system_hours_list[i]) for i in range(len(user_hours_list))]

        summary_text = f"The weekly average of Your studying hours is: {user_avg}\n\n"
        summary_text += avg_result + "\n\n"

        for i, (key, value) in enumerate(self.subject_dict[grade_input-1].items()):
            summary_text += f"{i+1}. {key}: {summary_results[i]}\n"

        messagebox.showinfo("Summary Report", summary_text)
        
        self.save_to_database(grade_input, user_hours_list, user_avg, system_avg, avg_result, summary_results)
        self.plot_summary(user_hours_list, system_hours_list, list(self.subject_dict[grade_input-1].keys()))

    def save_to_database(self, grade, user_hours, user_avg, system_avg, avg_result, summary_results):
        conn = sqlite3.connect('study_hours.db')
        c = conn.cursor()
        
        subjects = list(self.subject_dict[grade-1].keys())
        system_hours = list(self.subject_dict[grade-1].values())
        
        for i, subject in enumerate(subjects):
            c.execute("INSERT INTO StudyRecords (user_id, grade, subject, user_hours, system_hours, user_avg, system_avg, avg_result, summary_result) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                      (self.user_id, grade, subject, user_hours[i], system_hours[i], user_avg, system_avg, avg_result, summary_results[i]))
        
        conn.commit()
        conn.close()

    def plot_summary(self, user_hours, system_hours, subjects):
        plt.figure(figsize=(10, 5))
        bar_width = 0.35
        index = range(len(subjects))
        
        plt.bar(index, user_hours, bar_width, label='Your Hours')
        plt.bar([i + bar_width for i in index], system_hours, bar_width, label='System Hours')
        
        plt.xlabel('Subjects')
        plt.ylabel('Hours')
        plt.title('Weekly Study Hours')
        plt.xticks([i + bar_width / 2 for i in index], subjects, rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()

    def show_welcome_message(self):
        welcome_text = (
            "Welcome to the Study Hours Tracker!\n\n"
            "This application allows you to track your weekly study hours for various subjects. "
            "You can compare your study hours with the recommended hours and receive personalized feedback.\n\n"
            "To get started, please log in or register an account."
        )
        messagebox.showinfo("Welcome", welcome_text)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        conn = sqlite3.connect('study_hours.db')
        c = conn.cursor()
        c.execute("SELECT user_id FROM Users WHERE username=? AND password=?", (username, password))
        user = c.fetchone()
        
        if user:
            self.user_id = user[0]
            messagebox.showinfo("Login Success", "Successfully logged in!")
            self.show_main()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password.")
        
        conn.close()

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        conn = sqlite3.connect('study_hours.db')
        c = conn.cursor()
        c.execute("INSERT INTO Users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        conn.close()
        
        messagebox.showinfo("Registration Success", "Successfully registered! Please log in.")

    def logout(self):
        self.user_id = None
        self.username_entry.delete(0, tk.END)
        self.password_entry.delete(0, tk.END)
        self.show_login()

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = StudyHoursTracker(root)
    root.mainloop()
